<a href="https://colab.research.google.com/github/TimkaThymineTimiryazevskaya/sf_data_science/blob/main/HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [ ]:
data = pd.read_csv('_train_sem09__1_.zip')

In [ ]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

**Логистическая регрессия**

In [ ]:
log_reg = linear_model.LogisticRegression(max_iter = 50)
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Вывод**: Без подбора гиперпараметров модель лог.регрессии дает f1-score 0.79

In [ ]:
rf = ensemble.RandomForestClassifier(random_state=42)

rf.fit(X_train, y_train)
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Test: 0.81


**Вывод**: Без подбора гиперпараметров модель случайного леса дает f1-score 0.81

**Подбор гиперпараметров для лог.регрессии с помощью GridSeachCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'saga'], 
              }
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, 
        max_iter=50 
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: user 4.05 s, sys: 209 ms, total: 4.26 s
Wall time: 41.7 s
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'saga'}


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


**Подбор гиперпараметров для случайного леса с помощью GridSeachCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'min_samples_leaf': list(np.linspace(5, 30, 10, dtype=int)),
              'max_depth': list(np.linspace(5, 15, 30, dtype=int)),
              'criterion':['entropy','gini']
              }
grid_search = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: user 26.7 s, sys: 2.56 s, total: 29.2 s
Wall time: 39min 19s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 5}


**Вывод: Для случайного леса f1_score увеличился **

**Подбор гиперпараметров для лог.регрессии с помощью RandomizedSeachCV**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'saga'], 
              }
rand_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, 
        max_iter=50 
    ), 
    param_distributions=param_grid, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: user 22.9 s, sys: 1.94 s, total: 24.8 s
Wall time: 37min 57s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 5}


**Подбор гиперпараметров для случайного леса с помощью RandomizedSeachCV**

In [21]:
param_grid = {'min_samples_leaf': list(np.linspace(5, 100, 50, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 50, dtype=int)),
              'criterion':['entropy','gini']
              }
rand_search = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(
        random_state=42
    ), 
    param_distributions=param_grid, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: user 18.5 s, sys: 1.45 s, total: 20 s
Wall time: 37min 2s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 13, 'min_samples_leaf': 5}


Вывод: В обоих случаях f1 увеличился

**Подбор гиперпараметров для лог.регрессии с помощью Hyperopt**

In [22]:
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

In [26]:
space_lin={'penalty': hp.choice('penalty',options=['l2', 'none'] ),'solver': hp.choice('solver',options=['lbfgs', 'saga']), 
              }

In [35]:
def hyperopt_log(params, cv=5, X=X_train, y=y_train, random_state=42):
    params = {'penalty': (params['penalty']), 
              'solver': (params['solver']), 
              }
    model = linear_model.LogisticRegression (**params, random_state=42, max_iter=100)

    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    return -score

In [36]:
%%time

trials = Trials()

best=fmin(hyperopt_log, 
          space=space_lin, 
          algo=tpe.suggest,
          max_evals=20, 
          trials=trials, 
          rstate=np.random.RandomState(42)
         )
print("Наилучшие значения гиперпараметров {}".format(best))

  5%|▌         | 1/20 [00:09<02:56,  9.32s/it, best loss: -0.8720930232558138]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



 10%|█         | 2/20 [00:10<01:23,  4.62s/it, best loss: -0.9102053325160895]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 15%|█▌        | 3/20 [00:12<00:57,  3.37s/it, best loss: -0.9102053325160895]

...

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



 95%|█████████▌| 19/20 [01:23<00:05,  5.43s/it, best loss: -0.9102053325160895]

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



100%|██████████| 20/20 [01:25<00:00,  4.27s/it, best loss: -0.9102053325160895]
Наилучшие значения гиперпараметров {'penalty': 1, 'solver': 0}
CPU times: user 1min 22s, sys: 6.59 s, total: 1min 29s
Wall time: 1min 25s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



In [30]:
model = linear_model.LogisticRegression(
    random_state=42, 
    penalty=(best['penalty']),
    solver=(best['solver']),
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

ValueError: ignored

**Подбор гиперпараметров для случайного леса с помощью Hyperopt**

In [43]:
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [44]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    model = ensemble.RandomForestClassifier(**params, random_state=42)

    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    return -score

In [48]:
%%time

trials = Trials()

best=fmin(hyperopt_rf, 
          space=space, 
          algo=tpe.suggest,
          max_evals=20, 
          trials=trials, 
          rstate=np.random.RandomState(42)
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [01:01<00:00,  3.09s/it, best loss: -0.9911233547597184]
Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}
CPU times: user 58.7 s, sys: 683 ms, total: 59.4 s
Wall time: 1min 1s


In [50]:
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


**Подбор гиперпараметров для случайного леса с помощью Optuna**

In [51]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
import optuna

In [53]:
def optuna_rf(trial):
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [54]:
%%time
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
study.optimize(optuna_rf, n_trials=20)

[I 2023-02-11 17:38:44,654] A new study created in memory with name: RandomForestClassifier
[I 2023-02-11 17:38:46,765] Trial 0 finished with value: 0.9057527539779682 and parameters: {'n_estimators': 107, 'max_depth': 11, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9057527539779682.
[I 2023-02-11 17:38:50,362] Trial 1 finished with value: 0.9278287461773701 and parameters: {'n_estimators': 171, 'max_depth': 20, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.9278287461773701.
[I 2023-02-11 17:38:54,783] Trial 2 finished with value: 0.9525835866261398 and parameters: {'n_estimators': 112, 'max_depth': 12, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.9525835866261398.
[I 2023-02-11 17:38:59,871] Trial 3 finished with value: 0.9087022900763359 and parameters: {'n_estimators': 143, 'max_depth': 13, 'min_samples_leaf': 8}. Best is trial 2 with value: 0.9525835866261398.
[I 2023-02-11 17:39:05,757] Trial 4 finished with value: 0.9172013443324166 and parameters: {'n_

CPU times: user 1min 2s, sys: 706 ms, total: 1min 3s
Wall time: 1min 11s


In [55]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=42 )
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


Итог: При наилучших параметрах f1-score составил 0ю82 для тестововй выборки